In [10]:
import pandas as pd
import numpy as np

import params
import wandb
wandb.login()

from sklearn.preprocessing import LabelEncoder

In [11]:
df_fertilizer = pd.read_csv("../data/fertilizer_prediction.csv")
df_crop = pd.read_csv("../data/crop_recommendation.csv")

In [12]:
def process_fertilizer_df(df_fertilizer):
    soil_type= np.array(df_fertilizer['Soil Type'].values)
    crop_type= np.array(df_fertilizer['Crop Type'].values)

    label_encoder = LabelEncoder()
    integer_encoded_soil = label_encoder.fit_transform(soil_type)
    integer_encoded_crop = label_encoder.fit_transform(crop_type)

    dict_soil={}
    for i in range(len(integer_encoded_soil)):
        dict_soil[soil_type[i]]=integer_encoded_soil[i]
    dict_crop={}
    for i in range(len(integer_encoded_crop)):
        dict_crop[crop_type[i]]=integer_encoded_crop[i]

    df_fertilizer['Soil Type']=df_fertilizer['Soil Type'].map(dict_soil)
    df_fertilizer['Crop Type']=df_fertilizer['Crop Type'].map(dict_crop)

    df_fertilizer.drop(columns=["Humidity "], axis=1, inplace=True)

    return df_fertilizer

In [13]:
df_fertilizer = process_fertilizer_df(df_fertilizer)
df_fertilizer.head()

,Temparature,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,26,38,4,3,37,0,0,Urea
1,29,45,2,8,12,0,36,DAP
2,34,62,0,1,7,9,30,14-35-14
3,32,34,3,9,22,0,20,28-28
4,28,46,1,6,35,0,0,Urea


In [14]:
fertilizer_table = wandb.Table(dataframe=df_fertilizer)
crop_table = wandb.Table(dataframe=df_crop)

In [15]:
run = wandb.init(project=params.PROJECT, entity=params.ENTITY, job_type="data_upload")
data_artifact = wandb.Artifact(params.DATA_ART, type="data_upload")

In [16]:
data_artifact.add(fertilizer_table, "fertilizer_data_table")
data_artifact.add(crop_table, "crop_data_table")

ArtifactManifestEntry(path='crop_data_table.table.json', digest='JOa9X6QrATXENK1X/lCyZg==', ref=None, birth_artifact_id=None, size=175176, extra={}, local_path='C:\\Users\\ariha\\AppData\\Local\\wandb\\wandb\\artifacts\\staging\\tmpb9v4lgoz')

In [17]:
run.log_artifact(data_artifact)
run.finish()